In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


df_train = pd.read_csv("/kaggle/input/the-toxicity-prediction-challenge/train.csv")
df_feamat = pd.read_csv("/kaggle/input/the-toxicity-prediction-challenge/feamat.csv")
df_test = pd.read_csv("/kaggle/input/the-toxicity-prediction-challenge/test.csv")




In [ ]:
y = df_train["Expected"].to_numpy()

#split the chemical id and assay id in train.csv
df_train[['ChemId','AssayId']] = df_train.Id.str.split(";",expand=True) 
df_test[['ChemId','AssayId']] = df_test.x.str.split(";",expand=True) 


In [ ]:
#To do the mapping b/w df_train and feamat
df_feamat.rename(columns = {'V1':'ChemId'}, inplace = True) 

df_feamat = df_feamat.loc[:, (~df_feamat.isin([0])).any(axis = 0)]
df_feamat.replace([np.inf, -np.inf], np.nan, inplace =True)
df_feamat['V15'].fillna(value=df_feamat['V15'].mean(), inplace = True)

# df_train = df_train.merge(df_feamat, left_on='ChemId', right_on='ChemId', how='left')
# df_test = df_test.merge(df_feamat, left_on='ChemId', right_on='ChemId', how='left')

df_train = df_train.merge(df_feamat, on="ChemId", how="left")
df_test = df_test.merge(df_feamat, on="ChemId", how="left")
x = df_train.drop(['ChemId', 'Id','V2', 'Expected'], axis = 1).to_numpy()
target = df_test.drop(['ChemId','x', 'V2'], axis = 1).to_numpy()



df_train['AssayId'] = df_train.AssayId.astype(int)
df_test['AssayId'] = df_test.AssayId.astype(int)




In [ ]:
df_feamat.head()

In [ ]:

folds = StratifiedKFold(n_splits = 10,random_state = None, shuffle = False) 



In [ ]:
for train_index, test_index in folds.split(x,y):
    x_train, x_test, y_train, y_test = x[train_index], x[test_index], y[train_index], y[test_index]
    



In [ ]:

model = XGBClassifier(n_estimators=100, max_depth=3)
model.fit(x_train, y_train)
cv= cross_val_score(model,x_train,y_train,cv=folds)
print(cv)



In [ ]:
df_train.columns

In [ ]:
model = XGBClassifier()
model.fit(x_train, y_train)
cv= cross_val_score(model,x_train,y_train,cv=folds)
print(cv)

In [ ]:
print(cv.mean())

In [ ]:
model = XGBClassifier(n_estimators = 400, max_depth=8)
model.fit(x_train, y_train)
cv= cross_val_score(model,x_train,y_train,cv=folds)
print(cv.mean())

In [ ]:
preds = model.predict(target)
output_file = pd.DataFrame({'Id':  df_test.x, 'Predicted': preds})
print(output_file)
output_file.to_csv('XGBOostRRwithnestimator.csv', index=False)
print("Ouput file created")
